In [1]:
from test import EigenData
import numpy as np
import scipy.io
import h5py
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import torch
%matplotlib inline

In [2]:
data = EigenData()
eigenworms = data.get_eigenworms('EigenWorms.mat')

footage = data.get_footage('20150814-All-PNAS2011-DataStitched .mat')
newdata = torch.rand(33600,5)
for k in footage.keys():
    print(k, footage[k].shape)
    arr1=footage[k]

    arr2=arr1[:5,]

    r = data.reconstruct(footage[k])
    print('-->', r.shape)
    dataset1 = arr2.transpose()

    newdata = np.concatenate((newdata,dataset1),axis=0)
newdata = newdata[33600:,]
print(newdata.shape)

b (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
c (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
d (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
e (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
f (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
g (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
h (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
i (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
j (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
k (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
l (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
m (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
(403200, 5)


In [3]:
data1=newdata[~np.isnan(newdata).any(axis=1)]

data1 = data1.astype('float32')
print(data1.shape)
print(len(data1))

(386422, 5)
386422


In [4]:
def create_dataset(data,n_predictions,n_next):
    
   
    dim = data.shape[1]
    train_X, train_Y = [], []
    for i in range(data.shape[0]-n_predictions-n_next-1):
        a = data[i:(i+n_predictions),:]
        train_X.append(a)
        tempb = data[(i+n_predictions):(i+n_predictions+n_next),:]
        b = []
        for j in range(len(tempb)):
            for k in range(dim):
                b.append(tempb[j,k])
        train_Y.append(b)
    train_X = np.array(train_X,dtype='float64')
    train_Y = np.array(train_Y,dtype='float64')

    return train_X, train_Y

In [5]:
X, Y = create_dataset(data1,5,6)

print(X.shape)
print(Y.shape)

(386410, 5, 5)
(386410, 30)


In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))
data1 = scaler.fit_transform(data1)
Y = scaler.fit_transform(Y)

In [7]:
data_X, data_Y = create_dataset(data1,5,6)

print(data_X.shape)
print(data_Y.shape)

(386410, 5, 5)
(386410, 30)


In [8]:
train_size = int(len(data_X) * 0.8)
test_size = len(data_X) - train_size
train_X = data_X[:train_size]
train_Y = Y[:train_size]
test_X = data_X[train_size:]
test_Y = Y[train_size:]

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

(309128, 5, 5)
(309128, 30)
(77282, 5, 5)
(77282, 30)


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input,LSTM,Dropout,Dense


timesteps = 5
# create and fit the LSTM network

model = keras.Sequential()
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(30,activation='linear'))
model.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(train_X, train_Y, epochs=100, batch_size=512, verbose=2, validation_data=(test_X, test_Y))

Epoch 1/100
604/604 - 31s - loss: 0.0138 - root_mean_squared_error: 0.1174 - val_loss: 0.0048 - val_root_mean_squared_error: 0.0693
Epoch 2/100
604/604 - 26s - loss: 0.0055 - root_mean_squared_error: 0.0740 - val_loss: 0.0023 - val_root_mean_squared_error: 0.0484
Epoch 3/100
604/604 - 26s - loss: 0.0034 - root_mean_squared_error: 0.0579 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0412
Epoch 4/100
604/604 - 26s - loss: 0.0025 - root_mean_squared_error: 0.0501 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0375
Epoch 5/100
604/604 - 26s - loss: 0.0021 - root_mean_squared_error: 0.0455 - val_loss: 0.0010 - val_root_mean_squared_error: 0.0322
Epoch 6/100
604/604 - 26s - loss: 0.0018 - root_mean_squared_error: 0.0425 - val_loss: 0.0011 - val_root_mean_squared_error: 0.0333
Epoch 7/100
604/604 - 26s - loss: 0.0016 - root_mean_squared_error: 0.0402 - val_loss: 9.9067e-04 - val_root_mean_squared_error: 0.0315
Epoch 8/100
604/604 - 26s - loss: 0.0015 - root_mean_squared_error: 0.03

Epoch 60/100
604/604 - 26s - loss: 8.1395e-04 - root_mean_squared_error: 0.0285 - val_loss: 4.3892e-04 - val_root_mean_squared_error: 0.0210
Epoch 61/100
604/604 - 26s - loss: 8.1200e-04 - root_mean_squared_error: 0.0285 - val_loss: 4.3622e-04 - val_root_mean_squared_error: 0.0209
Epoch 62/100
604/604 - 26s - loss: 8.1153e-04 - root_mean_squared_error: 0.0285 - val_loss: 4.6185e-04 - val_root_mean_squared_error: 0.0215
Epoch 63/100
604/604 - 26s - loss: 8.1149e-04 - root_mean_squared_error: 0.0285 - val_loss: 4.4560e-04 - val_root_mean_squared_error: 0.0211
Epoch 64/100
604/604 - 26s - loss: 8.0699e-04 - root_mean_squared_error: 0.0284 - val_loss: 4.2402e-04 - val_root_mean_squared_error: 0.0206
Epoch 65/100
604/604 - 26s - loss: 8.0599e-04 - root_mean_squared_error: 0.0284 - val_loss: 4.4172e-04 - val_root_mean_squared_error: 0.0210
Epoch 66/100
604/604 - 26s - loss: 8.0369e-04 - root_mean_squared_error: 0.0283 - val_loss: 4.2467e-04 - val_root_mean_squared_error: 0.0206
Epoch 67/100


In [10]:
testPredict = model.predict(test_X)

In [11]:
newtestPredict = scaler.inverse_transform(testPredict)
newtest_Y = scaler.inverse_transform(test_Y)

In [12]:
testScore = sqrt(mean_squared_error(newtest_Y, newtestPredict))
print('Test Score: %.5f RMSE' % (testScore))

Test Score: 0.54975 RMSE
